### Load image and label

In [27]:
import os

path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\001\glasses'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if ".avi" in file:
            files.append(os.path.join(r, file))
        
for f in files:
    print(f)

.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\nonsleepyCombination.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\sleepyCombination.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\slowBlinkWithNodding.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\yawning.avi


### Crop image and Label
[HAAR Classifier](http://blog.topspeedsnail.com/archives/10511)
- haarcascade_frontalface_default(想法: 直接用這個切出臉部的框框)
- haarcascade_eye
- problem: 多個人臉的問題防範

In [26]:
import numpy as np
import cv2

filename = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\001\glasses\yawning.avi'
cap = cv2.VideoCapture(filename)

while(cap.isOpened()):
    ret, frame = cap.read()
    #print(type(frame))
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                
    cv2.imshow('frame',frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

In [23]:
import cv2
import sys

img = cv2.imread('.\Temp\people.jpg')

# 加载分类器
face_haar = cv2.CascadeClassifier(".\Tool\Detecter\haarcascade_frontalface_default.xml")
eye_haar = cv2.CascadeClassifier(".\Tool\Detecter\haarcascade_eye.xml")
mouth_haar = cv2.CascadeClassifier(".\Tool\Detecter\haarcascade_mcs_mouth.xml")
nose_haar = cv2.CascadeClassifier(".\Tool\Detecter\haarcascade_mcs_nose.xml")

# 把图像转为黑白图像
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 检测图像中的所有脸
faces = face_haar.detectMultiScale(gray_img, 1.3, 5)
for face_x,face_y,face_w,face_h in faces:
    cv2.rectangle(img, (face_x, face_y), (face_x+face_w, face_y+face_h), (0,255,0), 2)
        # 眼长在脸上
    roi_gray_img = gray_img[face_y:face_y+face_h, face_x:face_x+face_w]
    roi_img = img[face_y:face_y+face_h, face_x:face_x+face_w]
    
    eyes = eye_haar.detectMultiScale(roi_gray_img, 1.3, 5)
    for eye_x,eye_y,eye_w,eye_h in eyes:
        cv2.rectangle(roi_img, (eye_x,eye_y), (eye_x+eye_w, eye_y+eye_h), (255,0,0), 2)
    
    noses = nose_haar.detectMultiScale(roi_gray_img, 1.7, 5)
    for nose_x,nose_y,nose_w,nose_h in noses:
        cv2.rectangle(roi_img, (nose_x,nose_y), (nose_x+nose_w, nose_y+nose_h), (255,0,255), 2)
    
    mouthes = mouth_haar.detectMultiScale(roi_gray_img, 1.5, 7)
    for mouth_x,mouth_y,mouth_w,mouth_h in mouthes:
        cv2.rectangle(roi_img, (mouth_x,mouth_y), (mouth_x+mouth_w, mouth_y+mouth_h), (255,255,0), 2)
    

cv2.imshow('img', img)
cv2.waitKey(0)

cv2.destroyAllWindows()